In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import matplotlib as plt
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from sklearn.preprocessing import LabelEncoder, StandardScaler

C:\Users\Rafi\AppData\Local\Temp\ipykernel_20588\1464394444.py:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# Data Preprocessing

In [5]:
# Load and preprocess the data
data = pd.read_csv('nutrition.csv')
data.rename(columns={'name': 'label'}, inplace=True)

# Extract features and labels
x = data[['calories', 'proteins', 'fat', 'carbohydrate']]
y = data['label']

# Normalize the features
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

# Encode the labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Convert to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_scaled, y_encoded)).batch(64)


# Model Building

In [3]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(units=hp.Int('units1', min_value=32, max_value=256, step=32),
                                    activation='relu', input_shape=(x_scaled.shape[1],)))
    model.add(tf.keras.layers.Dense(units=hp.Int('units2', min_value=16, max_value=256, step=32),
                                    activation='relu'))
    model.add(tf.keras.layers.Dense(units=hp.Int('units3', min_value=16, max_value=256, step=32),
                                    activation='relu'))
    model.add(tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax'))

    model.compile(optimizer=hp.Choice('optimizer', ['adam', 'rmsprop']),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [10]:
class PlotTrainingCurve(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.metrics = {}

    def on_epoch_end(self, epoch, logs={}):
        for metric, value in logs.items():
            self.metrics.setdefault(metric, []).append(value)

plot_training_curve = PlotTrainingCurve()

In [4]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>=0.9):
            print("\nDesired accuracy reached, training stopped.")
            self.model.stop_training = True

# Hyperparameter Tuning

In [7]:
tuner = RandomSearch(
    build_model,
    objective='accuracy',
    max_trials=5,
    executions_per_trial=1,
    directory='my_tuning_dir',
    project_name='nutrition_model_tuning')

tuner.search(x=x_scaled, y=y_encoded, epochs=1000)
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Reloading Tuner from my_tuning_dir\nutrition_model_tuning\tuner0.json


In [ ]:
def plot_metrics(history, metric):
    plt.plot(history.metrics[metric], label=metric)
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.show()

plot_metrics(plot_training_curve, 'accuracy')

In [8]:
callback = myCallback()
hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(train_dataset, epochs=1500, callbacks=[callback])

Epoch 1/1500
22/22 [==============================] - 2s 7ms/step - loss: 7.2188 - accuracy: 7.4294e-04
Epoch 2/1500
22/22 [==============================] - 0s 6ms/step - loss: 7.1736 - accuracy: 0.0037
Epoch 3/1500
22/22 [==============================] - 0s 6ms/step - loss: 6.8829 - accuracy: 0.0037
Epoch 4/1500
22/22 [==============================] - 0s 7ms/step - loss: 6.1874 - accuracy: 0.0022
Epoch 5/1500
22/22 [==============================] - 0s 7ms/step - loss: 5.7294 - accuracy: 0.0037
Epoch 6/1500
22/22 [==============================] - 0s 7ms/step - loss: 5.4727 - accuracy: 0.0059
Epoch 7/1500
22/22 [==============================] - 0s 7ms/step - loss: 5.2500 - accuracy: 0.0111
Epoch 8/1500
22/22 [==============================] - 0s 7ms/step - loss: 5.0721 - accuracy: 0.0126
Epoch 9/1500
22/22 [==============================] - 0s 6ms/step - loss: 4.9736 - accuracy: 0.0149
Epoch 10/1500
22/22 [==============================] - 0s 6ms/step - loss: 4.8207 - accuracy: 0.

In [9]:
hypermodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               1280      
                                                                 
 dense_1 (Dense)             (None, 240)               61680     
                                                                 
 dense_2 (Dense)             (None, 176)               42416     
                                                                 
 dense_3 (Dense)             (None, 1346)              238242    
                                                                 
Total params: 343618 (1.31 MB)
Trainable params: 343618 (1.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Membuat Prediksi

In [14]:
def predict_food(calories, proteins, fat, carbohydrate):
    input_data = np.array([[calories, proteins, fat, carbohydrate]])
    input_data_scaled = scaler.transform(input_data)
    predictions = hypermodel.predict(input_data_scaled)
    predicted_label = label_encoder.inverse_transform([np.argmax(predictions)])
    return predicted_label[0]

input_nutrients = [25, 1.6, 0.2, 5.3]
predicted_food = predict_food(*input_nutrients)
print(f"Input nutrients: {input_nutrients} -> Predicted food: {predicted_food}")

1/1 [==============================] - 0s 20ms/step
Input nutrients: [25, 1.6, 0.2, 5.3] -> Predicted food: Andewi


C:\Users\Rafi\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [15]:
file_name = 'model.h5'
keras.models.save_model(hypermodel, file_name)

C:\Users\Rafi\AppData\Local\Temp\ipykernel_9728\3176766029.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  keras.models.save_model(hypermodel, file_name)


In [16]:
from keras.models import load_model
save_model = load_model('model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(save_model)
tflite_model = converter.convert()
open("food_prediction_model.tflite", "wb").write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\Rafi\AppData\Local\Temp\tmp2kv_cw6k\assets


INFO:tensorflow:Assets written to: C:\Users\Rafi\AppData\Local\Temp\tmp2kv_cw6k\assets


1517060